In [1]:
import pandas as pd
import numpy as np
import re
import subprocess
import warnings

from Bio import SeqIO, AlignIO, motifs
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqUtils.ProtParam import ProteinAnalysis, ProtParamData
from Bio.Align import AlignInfo
from Bio import BiopythonDeprecationWarning
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import seaborn as sns

# Suppress specific Biopython deprecation warnings
warnings.filterwarnings('ignore', category=BiopythonDeprecationWarning)


In [2]:
# Load the dataset
lysin_df = pd.read_json('lysin_unnested_data.json')

In [3]:
lysin_df.fillna("Unknown", inplace=True)

In [4]:
lysin_df.head()

,GeneID,phams,Start,Stop,Length,Name,translation,Orientation,Notes,PhageID,Accession,HostStrain,Cluster
0,20ES_CDS_10,[152741],6442,7420,978,20ES,MSLQVGSSGELVNRWIRVMKARFASYAGKLKEDGYFGLDDKAVQQE...,F,b'lysin B',20ES,KJ410132,Mycobacterium,A2
1,20ES_CDS_8,[152812],4689,5988,1299,20ES,MTAVITRKQAQWVHDMARARNGLPYAYGGAFTNDPKRSTDCSGLVL...,F,b'lysin A',20ES,KJ410132,Mycobacterium,A2
2,244_CDS_34,[152812],29920,31423,1503,244,MSVTRANVEATKRFIGERVGNPYVYGGALSPTNVHQGTDCSEVWQT...,F,b'lysin A',244,DQ398041,Mycobacterium,E
3,32HC_CDS_36,[152814],27404,28772,1368,32HC,MPGSEIPRYWPLGAGRIVTSPFGPRSGGFHAGVDFGRNGGSAGMPV...,F,b'lysin A',32HC,KJ028219,Mycobacterium,Z
4,32HC_CDS_37,[3452],28786,29938,1152,32HC,MAWKQPQLTDPPMVSEEIGKLNRRLLLAYAANSRAVEAGVQLHDVF...,F,b'lysin B',32HC,KJ028219,Mycobacterium,Z


In [5]:
# Define the search patterns for each lysin type
lysin_A_patterns = [
    'lysin A', 'putative lysin A', 'lysin A, protease M23 domain',
    'lysin A, N-acetylmuramoyl-L-alanine amidase domain', 'lysin A, L-Ala-D-Glu peptidase domain',
    'lysin A, glycosyl hydrolase domain', 'lysin A, protease C39 domain',
    'lysin A, N-acetylmuramoyl-L-alanine amidase', 'lysin A, M23 peptidase domain',
    'lysin A, amidase domain', 'lysin A, L-ala D-glu peptidase domain',
    'lysin A glycosidase domain', 'lysin A amidase domain', 'lysin A peptidase domain',
    'lysin A, N-acetylmuramoyl-L-alanine amides domain', 'lysin A, protease M15 domain',
    'lysin A N-acetylmuramoyl-L-alanine amidase domain', 'lysin A L-Ala-D-Glu peptidase domain',
    'lysin A, protease domain', 'lysin A, N-acetyl-B-D-muramidase domain',
    'lysin A glycosyl hydrolase domain', 'lysin A, GH19 glycoside hydrolase domain',
    'lysin A, N-acetyl-beta-D-muramidase domain', 'lysin A, peptidase domain',
    'lysinA, hydrolase domain', 'Lysin A', 'lysin A, N-acetylmuramoyl-Lalanine amidase domain',
    'lysin A, l-ala-d-glu peptidase domain', 'lysin A, C39 peptidase domain',
    'lysin A, glycosyl hyrdrolase domain', 'lysin A, L-Ala,-D-Glu peptidase domain',
    'lysin A, M15 protease', 'lysin A, protease C39', 'putitive lysin A'
]

lysin_B_patterns = [
    'lysin B', 'putative lysin B', 'lysinB protein', 'putitive lysin B'
]

endolysin_patterns = [
    'endolysin', 'LysM-like endolysin', 'endolysin, L-Ala-D-Glu peptidase domain',
    'endolysin, N-acetylmuramoyl-L-alanine amidase domain', 'endolysin, protease M23 domain',
    'endolysin, protease C39 domain', 'endolysin, glycosyl hydrolase domain',
    'endolysin domain protein', 'endolysin, protease M15 domain', 'endolysin amidase',
    'endolysin endopeptidase/amidase', 'putative endolysin', 'endolysin, protease M23 domain and cell wall binding domain',
    'endolysin, protease domain'
]

# Combine all patterns into regex patterns
lysin_A_pattern = '|'.join(lysin_A_patterns)
lysin_B_pattern = '|'.join(lysin_B_patterns)
endolysin_pattern = '|'.join(endolysin_patterns)

# Define a function to determine the LysinType
def determine_lysin_type(notes):
    if re.search(lysin_A_pattern, notes, re.IGNORECASE):
        return 'lysin_A'
    elif re.search(lysin_B_pattern, notes, re.IGNORECASE):
        return 'lysin_B'
    elif re.search(endolysin_pattern, notes, re.IGNORECASE):
        return 'endolysin'
    else:
        return 'unknown'

# Apply the function to create a new column LysinType
lysin_df['LysinType'] = lysin_df['Notes'].apply(determine_lysin_type)

In [6]:
lysin_df.head()

,GeneID,phams,Start,Stop,Length,Name,translation,Orientation,Notes,PhageID,Accession,HostStrain,Cluster,LysinType
0,20ES_CDS_10,[152741],6442,7420,978,20ES,MSLQVGSSGELVNRWIRVMKARFASYAGKLKEDGYFGLDDKAVQQE...,F,b'lysin B',20ES,KJ410132,Mycobacterium,A2,lysin_B
1,20ES_CDS_8,[152812],4689,5988,1299,20ES,MTAVITRKQAQWVHDMARARNGLPYAYGGAFTNDPKRSTDCSGLVL...,F,b'lysin A',20ES,KJ410132,Mycobacterium,A2,lysin_A
2,244_CDS_34,[152812],29920,31423,1503,244,MSVTRANVEATKRFIGERVGNPYVYGGALSPTNVHQGTDCSEVWQT...,F,b'lysin A',244,DQ398041,Mycobacterium,E,lysin_A
3,32HC_CDS_36,[152814],27404,28772,1368,32HC,MPGSEIPRYWPLGAGRIVTSPFGPRSGGFHAGVDFGRNGGSAGMPV...,F,b'lysin A',32HC,KJ028219,Mycobacterium,Z,lysin_A
4,32HC_CDS_37,[3452],28786,29938,1152,32HC,MAWKQPQLTDPPMVSEEIGKLNRRLLLAYAANSRAVEAGVQLHDVF...,F,b'lysin B',32HC,KJ028219,Mycobacterium,Z,lysin_B


In [7]:
# Check for missing values in each column
missing_values = lysin_df.isnull().sum()
missing_values

GeneID         0
phams          0
Start          0
Stop           0
Length         0
Name           0
translation    0
Orientation    0
Notes          0
PhageID        0
Accession      0
HostStrain     0
Cluster        0
LysinType      0
dtype: int64

In [8]:
# Check data types and non-null counts
lysin_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7182 entries, 0 to 7181
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   GeneID       7182 non-null   object
 1   phams        7182 non-null   object
 2   Start        7182 non-null   int64 
 3   Stop         7182 non-null   int64 
 4   Length       7182 non-null   int64 
 5   Name         7182 non-null   object
 6   translation  7182 non-null   object
 7   Orientation  7182 non-null   object
 8   Notes        7182 non-null   object
 9   PhageID      7182 non-null   object
 10  Accession    7182 non-null   object
 11  HostStrain   7182 non-null   object
 12  Cluster      7182 non-null   object
 13  LysinType    7182 non-null   object
dtypes: int64(3), object(11)
memory usage: 785.7+ KB


In [9]:
# Get counts of each LysinType
lysin_type_counts = lysin_df['LysinType'].value_counts()
lysin_type_counts

LysinType
lysin_A      3458
lysin_B      2528
endolysin    1147
unknown        49
Name: count, dtype: int64

In [10]:
# Filter the dataframe by Lysin type B
lysin_b_df = lysin_df[lysin_df['LysinType'] == 'lysin_B']

In [11]:
lysin_b_df.head()

,GeneID,phams,Start,Stop,Length,Name,translation,Orientation,Notes,PhageID,Accession,HostStrain,Cluster,LysinType
0,20ES_CDS_10,[152741],6442,7420,978,20ES,MSLQVGSSGELVNRWIRVMKARFASYAGKLKEDGYFGLDDKAVQQE...,F,b'lysin B',20ES,KJ410132,Mycobacterium,A2,lysin_B
4,32HC_CDS_37,[3452],28786,29938,1152,32HC,MAWKQPQLTDPPMVSEEIGKLNRRLLLAYAANSRAVEAGVQLHDVF...,F,b'lysin B',32HC,KJ028219,Mycobacterium,Z,lysin_B
6,39HC_CDS_42,[152741],40378,41374,996,39HC,MSLALGSSGLMAAAWAAMMRLRFPGYALGRDGKPLGVDGYFGYDEE...,F,b'lysin B',39HC,KJ433973,Mycobacterium,B6,lysin_B
8,40AC_CDS_14,[152741],8492,9470,978,40AC,MSLTLGSQGEIVNRWIRVMKARFVSYAGKLREDAYFGLDDAEVQKE...,F,b'lysin B',40AC,KJ192196,Mycobacterium,A17,lysin_B
10,40BC_CDS_42,[152741],40378,41374,996,40BC,MSLALGSSGLMAAAWAAMMRLRFPGYALGRDGKPLGVDGYFGYDEE...,F,b'lysin B',40BC,KJ433975,Mycobacterium,B6,lysin_B


In [12]:
# Drop columns in lysin b df
lysin_b_df = lysin_b_df.drop(columns=['Length', 'Start', 'Stop', 'Orientation', 'Accession', 'Notes'])

# Function to clean the "phams" values
def clean_phams_value(value):
    # Extract the integer from the list
    return value[0]

# Apply the function to the "phams" column
lysin_b_df['phams'] = lysin_b_df['phams'].apply(clean_phams_value)

In [13]:
lysin_b_df.head()

,GeneID,phams,Name,translation,PhageID,HostStrain,Cluster,LysinType
0,20ES_CDS_10,152741,20ES,MSLQVGSSGELVNRWIRVMKARFASYAGKLKEDGYFGLDDKAVQQE...,20ES,Mycobacterium,A2,lysin_B
4,32HC_CDS_37,3452,32HC,MAWKQPQLTDPPMVSEEIGKLNRRLLLAYAANSRAVEAGVQLHDVF...,32HC,Mycobacterium,Z,lysin_B
6,39HC_CDS_42,152741,39HC,MSLALGSSGLMAAAWAAMMRLRFPGYALGRDGKPLGVDGYFGYDEE...,39HC,Mycobacterium,B6,lysin_B
8,40AC_CDS_14,152741,40AC,MSLTLGSQGEIVNRWIRVMKARFVSYAGKLREDAYFGLDDAEVQKE...,40AC,Mycobacterium,A17,lysin_B
10,40BC_CDS_42,152741,40BC,MSLALGSSGLMAAAWAAMMRLRFPGYALGRDGKPLGVDGYFGYDEE...,40BC,Mycobacterium,B6,lysin_B


In [14]:
# Function to calculate properties of lysinB protein
def calculate_properties(seq):
    analyzed_seq = ProteinAnalysis(seq)
    molecular_weight = analyzed_seq.molecular_weight()
    aromaticity = analyzed_seq.aromaticity()
    instability_index = analyzed_seq.instability_index()
    gravy = analyzed_seq.gravy()
    isoelectric_point = analyzed_seq.isoelectric_point()
    aliphatic_index = (analyzed_seq.count_amino_acids().get('A', 0) + 
                       2.9 * analyzed_seq.count_amino_acids().get('V', 0) + 
                       3.9 * (analyzed_seq.count_amino_acids().get('I', 0) + analyzed_seq.count_amino_acids().get('L', 0))) / len(seq) * 100
    kd_scale = ProtParamData.kd
    hydrophobicity_values = [kd_scale[aa] for aa in seq if aa in kd_scale]
    hydrophobicity = np.mean(hydrophobicity_values) if hydrophobicity_values else 0
    return {
        'Molecular_Weight': molecular_weight,
        'Aromaticity': aromaticity,
        'Instability_Index': instability_index,
        'Gravy': gravy,
        'Isoelectric_Point': isoelectric_point,
        'Aliphatic_Index': aliphatic_index,
        'Hydrophobicity': hydrophobicity,
        'Protein_Length': len(seq)
    }

In [15]:
# Apply the function to each row in the dataframe
properties_df = lysin_b_df['translation'].apply(calculate_properties).apply(pd.Series)

# Merge the calculated properties back to the original dataframe
combined_lysin_b_df = pd.concat([lysin_b_df, properties_df], axis=1)

In [16]:
combined_lysin_b_df.head()

,GeneID,phams,Name,translation,PhageID,HostStrain,Cluster,LysinType,Molecular_Weight,Aromaticity,Instability_Index,Gravy,Isoelectric_Point,Aliphatic_Index,Hydrophobicity,Protein_Length
0,20ES_CDS_10,152741,20ES,MSLQVGSSGELVNRWIRVMKARFASYAGKLKEDGYFGLDDKAVQQE...,20ES,Mycobacterium,A2,lysin_B,36578.3514,0.107692,28.285877,-0.326462,5.823575,82.584615,-0.326462,325.0
4,32HC_CDS_37,3452,32HC,MAWKQPQLTDPPMVSEEIGKLNRRLLLAYAANSRAVEAGVQLHDVF...,32HC,Mycobacterium,Z,lysin_B,41216.7621,0.091384,41.574413,0.020366,6.173590,89.947781,0.020366,383.0
6,39HC_CDS_42,152741,39HC,MSLALGSSGLMAAAWAAMMRLRFPGYALGRDGKPLGVDGYFGYDEE...,39HC,Mycobacterium,B6,lysin_B,35566.7986,0.084592,26.417221,-0.292447,6.453408,74.048338,-0.292447,331.0
8,40AC_CDS_14,152741,40AC,MSLTLGSQGEIVNRWIRVMKARFVSYAGKLREDAYFGLDDAEVQKE...,40AC,Mycobacterium,A17,lysin_B,36595.5658,0.104615,33.097846,-0.276308,6.379347,86.769231,-0.276308,325.0
10,40BC_CDS_42,152741,40BC,MSLALGSSGLMAAAWAAMMRLRFPGYALGRDGKPLGVDGYFGYDEE...,40BC,Mycobacterium,B6,lysin_B,35566.7986,0.084592,26.417221,-0.292447,6.453408,74.048338,-0.292447,331.0


In [17]:
# Function to detect outliers using a less stringent IQR method
def detect_outliers_iqr(data, multiplier=2):
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    return ((data < lower_bound) | (data > upper_bound))

# Apply the less stringent outlier detection function to numerical columns
numerical_columns = ['Molecular_Weight', 'Aromaticity', 'Instability_Index', 'Gravy', 'Isoelectric_Point', 'Aliphatic_Index', 'Hydrophobicity', 'Protein_Length']
outliers = combined_lysin_b_df[numerical_columns].apply(detect_outliers_iqr)

# Count of outliers in each feature
outliers.sum().sort_values(ascending=False)


Aromaticity          13
Instability_Index     7
Protein_Length        7
Molecular_Weight      6
Gravy                 0
Isoelectric_Point     0
Aliphatic_Index       0
Hydrophobicity        0
dtype: int64

In [18]:
# Create a mask to keep rows without outliers in any of the numerical columns
no_outliers_mask = ~outliers.any(axis=1)

# Filter the DataFrame to remove outliers
cleaned_lysin_b_df = combined_lysin_b_df[no_outliers_mask]

In [19]:
cleaned_lysin_b_df.describe()

,Molecular_Weight,Aromaticity,Instability_Index,Gravy,Isoelectric_Point,Aliphatic_Index,Hydrophobicity,Protein_Length
count,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000,2502.000000
mean,35184.263601,0.092105,34.591262,-0.161509,6.629431,85.000522,-0.161509,323.715028
std,6276.878948,0.011762,6.320897,0.190654,1.365119,7.181747,0.190654,63.448026
min,21611.236100,0.058419,16.612500,-0.487379,4.474843,65.454545,-0.487379,202.000000
25%,30829.628175,0.083333,30.743958,-0.318078,5.682956,79.298246,-0.318078,279.000000
50%,35961.657700,0.093093,34.021472,-0.205096,6.058661,84.577114,-0.205096,323.000000
75%,37787.245500,0.099415,39.262368,-0.026087,7.747433,91.875000,-0.026087,342.000000
max,49702.271800,0.129693,56.210638,0.213877,9.977272,107.490347,0.213877,467.000000


In [20]:
# Function to perform MSA and return alignment
def perform_msa(dataframe, output_prefix):
    sequences = [SeqRecord(Seq(row['translation']), id=row['GeneID']) for index, row in tqdm(dataframe.iterrows(), total=dataframe.shape[0], desc='Creating SeqRecord objects')]
    fasta_file = f"{output_prefix}_lysins.fasta"
    SeqIO.write(sequences, fasta_file, "fasta")
    print(f"Sequences written to {fasta_file}")
    aligned_file = f"aligned_{output_prefix}_lysins.fasta"
    clustal_omega_command = ["clustalo", "-i", fasta_file, "-o", aligned_file, "--force"]
    print(f"Running Clustal Omega for multiple sequence alignment on {output_prefix}...")
    subprocess.run(clustal_omega_command)
    print(f"Multiple sequence alignment completed and saved to {aligned_file}")
    alignment = AlignIO.read(aligned_file, "fasta")
    print(f"Alignment read from {aligned_file}")
    return alignment

# Function to print the consensus sequence
def print_consensus(consensus, alignment_name):
    print(f"Consensus Sequence for {alignment_name}:")
    consensus_str = str(consensus)
    for i in range(0, len(consensus_str), 80):
        print(consensus_str[i:i+80])
    print("\n")

# Perform MSA for the cleaned Lysin B DataFrame
alignment_lysin_b = perform_msa(cleaned_lysin_b_df, "lysin_b")


Creating SeqRecord objects: 100%|███████████████████████| 2502/2502 [00:00<00:00, 23470.70it/s]


Sequences written to lysin_b_lysins.fasta
Running Clustal Omega for multiple sequence alignment on lysin_b...
Multiple sequence alignment completed and saved to aligned_lysin_b_lysins.fasta
Alignment read from aligned_lysin_b_lysins.fasta


In [21]:
# Print the consensus sequence
summary_align = AlignInfo.SummaryInfo(alignment_lysin_b)
consensus = summary_align.dumb_consensus()
print_consensus(consensus, "Lysin B")

Consensus Sequence for Lysin B:
MSTPLWLRPSSVRALKAWSAVTRTAIRRTTASRISDGIPGRPTVWTQSGTELIRPSKRPTANTGTSSRPRTPWFSAESIA
DXXXXTXXXXMXXMRVMXXXXXXPVVGAPLRPLPPHRQMXXXXXXLXGGXGXXDANXXXXXXXXIIXPXVXXXXXXMXXX
XXXYAXXXXXXXXXXDXXXLXXVFXYXKHLKXTTYFGXXYXXXXXXXEHGXXXXXXXXXXXXXXXXXXQXRXXXXXXXGX
XXXXAPGXXXXXXXXXGXXXXXXXXXXXXXXXXTXGXXXXPXXXXXXXXXXXXXGTGXXDXXXXXXXXXXXXXXGXGFXX
XXXXXXXXXXXXXXXXXLXXXXXXXXXXTDXXXXXXXXXNPXFXXXXXXPXPXXXXPXXXXXXXXXXXXXXPSXXRSYQX
SXXXGXXXXXXXXXXXXXXXXXXXXAPGFXXXXXXXXXXXXXXXXXXXGYSXGAXXXXXXXXXXXXXXXXXGFTIPKTGE
TVGPGXLXXXXGXXXXXXVXXXXXXXXGXPXRXXGXXXXXXXXXXXXXXXXXLXXXXXXGIXXXVXXXXXXXXXVXXXAX
XXGXXXXXXXXXXXXXXXXXXXDXYXXXXXVPIXXXXYXXXXXXXXXRXXXXXXXXXXXXXXXXVXXXXXXXXWXAQGXD
XXXXXXXXXXXXXXDXXXXXXXQXXXXXXFNLEXGXXGLXLGLGGVGPXXXXXXXXXXXXGLLGGXLGGGLGGLGXGXXG
XAXPXALLAPGVGLGXXXXGXXXXXXXXXQXXXXXTSALPGLIAGVXGXXXXXPXXXXXAXXXAXXXXXXFXXXXXXXXX
YXXXXXXXXXIPGLXRXXXXXXXXXHXXYXXXXXXXXLXXXXXXXXXXXXXXXTXXHXXYXXXXXXXXXXXGXXXXXRLX
XXXXXXXXXXXXXXXAXXXXXXXXXSVXXXXXFDTAVRLQGDXI




In [22]:
# Function to calculate Hamming distance
def hamming_distance(seq1, seq2):
    return sum(a != b for a, b in zip(seq1, seq2) if a != '-' and b != '-')

# Calculate Hamming distances
hamming_distances = [
    hamming_distance(str(record.seq), str(consensus))
    for record in tqdm(alignment_lysin_b, desc="Calculating Hamming Distances")
]

# Create a DataFrame with the calculated Hamming distances
df_lysin_b_distances = pd.DataFrame({
    "GeneID": [record.id for record in alignment_lysin_b],
    "Hamming_Distance": hamming_distances
})

Calculating Hamming Distances: 100%|████████████████████| 2502/2502 [00:00<00:00, 15837.52it/s]


In [23]:
# Merge the original cleaned DataFrame with the new distances DataFrame
combined_lysin_b_df = cleaned_lysin_b_df.merge(df_lysin_b_distances, on='GeneID', how='inner')

In [24]:
combined_lysin_b_df.head()

,GeneID,phams,Name,translation,PhageID,HostStrain,Cluster,LysinType,Molecular_Weight,Aromaticity,Instability_Index,Gravy,Isoelectric_Point,Aliphatic_Index,Hydrophobicity,Protein_Length,Hamming_Distance
0,20ES_CDS_10,152741,20ES,MSLQVGSSGELVNRWIRVMKARFASYAGKLKEDGYFGLDDKAVQQE...,20ES,Mycobacterium,A2,lysin_B,36578.3514,0.107692,28.285877,-0.326462,5.823575,82.584615,-0.326462,325.0,284
1,32HC_CDS_37,3452,32HC,MAWKQPQLTDPPMVSEEIGKLNRRLLLAYAANSRAVEAGVQLHDVF...,32HC,Mycobacterium,Z,lysin_B,41216.7621,0.091384,41.574413,0.020366,6.173590,89.947781,0.020366,383.0,350
2,39HC_CDS_42,152741,39HC,MSLALGSSGLMAAAWAAMMRLRFPGYALGRDGKPLGVDGYFGYDEE...,39HC,Mycobacterium,B6,lysin_B,35566.7986,0.084592,26.417221,-0.292447,6.453408,74.048338,-0.292447,331.0,293
3,40AC_CDS_14,152741,40AC,MSLTLGSQGEIVNRWIRVMKARFVSYAGKLREDAYFGLDDAEVQKE...,40AC,Mycobacterium,A17,lysin_B,36595.5658,0.104615,33.097846,-0.276308,6.379347,86.769231,-0.276308,325.0,284
4,40BC_CDS_42,152741,40BC,MSLALGSSGLMAAAWAAMMRLRFPGYALGRDGKPLGVDGYFGYDEE...,40BC,Mycobacterium,B6,lysin_B,35566.7986,0.084592,26.417221,-0.292447,6.453408,74.048338,-0.292447,331.0,293


In [25]:
# Initialize MinMaxScaler
scaler = MinMaxScaler()

In [26]:
# Apply MinMaxScaler to the features and reverse scores where lower values are better
combined_lysin_b_df['Instability_Index_Score'] = 1 - scaler.fit_transform(combined_lysin_b_df[['Instability_Index']])
combined_lysin_b_df['Isoelectric_Point_Score'] = scaler.fit_transform(combined_lysin_b_df[['Isoelectric_Point']])
combined_lysin_b_df['Aliphatic_Index_Score'] = scaler.fit_transform(combined_lysin_b_df[['Aliphatic_Index']])
combined_lysin_b_df['Aromaticity_Score'] = scaler.fit_transform(combined_lysin_b_df[['Aromaticity']])
combined_lysin_b_df['Gravy_Score'] = scaler.fit_transform(combined_lysin_b_df[['Gravy']])
combined_lysin_b_df['Hydrophobicity_Score'] = scaler.fit_transform(combined_lysin_b_df[['Hydrophobicity']])
combined_lysin_b_df['Molecular_Weight_Score'] = 1 - scaler.fit_transform(combined_lysin_b_df[['Molecular_Weight']])
combined_lysin_b_df['Protein_Length_Score'] = 1 - scaler.fit_transform(combined_lysin_b_df[['Protein_Length']])
combined_lysin_b_df['Hamming_Distance_Score'] = 1 - scaler.fit_transform(combined_lysin_b_df[['Hamming_Distance']])

In [27]:
# Define groups including Aromaticity in stability
stability_features = ['Instability_Index_Score', 'Isoelectric_Point_Score', 'Aliphatic_Index_Score', 'Aromaticity_Score']
hydrophobicity_features = ['Gravy_Score', 'Hydrophobicity_Score']
size_features = ['Molecular_Weight_Score', 'Protein_Length_Score']
similarity_features = ['Hamming_Distance_Score']

In [28]:
# Calculate group scores
combined_lysin_b_df['Stability_Score'] = combined_lysin_b_df[stability_features].mean(axis=1)
combined_lysin_b_df['Hydrophobicity_Score'] = combined_lysin_b_df[hydrophobicity_features].mean(axis=1)
combined_lysin_b_df['Size_Score'] = combined_lysin_b_df[size_features].mean(axis=1)
combined_lysin_b_df['Similarity_Score'] = combined_lysin_b_df[similarity_features].mean(axis=1)

In [29]:
# Calculate the final Treatment Score as the average of group scores
group_scores = ['Stability_Score', 'Hydrophobicity_Score', 'Size_Score', 'Similarity_Score']
combined_lysin_b_df['Treatment_Score'] = combined_lysin_b_df[group_scores].mean(axis=1)

In [30]:
combined_lysin_b_df.head()

,GeneID,phams,Name,translation,PhageID,HostStrain,Cluster,LysinType,Molecular_Weight,Aromaticity,...,Aromaticity_Score,Gravy_Score,Hydrophobicity_Score,Molecular_Weight_Score,Protein_Length_Score,Hamming_Distance_Score,Stability_Score,Size_Score,Similarity_Score,Treatment_Score
0,20ES_CDS_10,152741,20ES,MSLQVGSSGELVNRWIRVMKARFASYAGKLKEDGYFGLDDKAVQQE...,20ES,Mycobacterium,A2,lysin_B,36578.3514,0.107692,...,0.691323,0.229470,0.229470,0.467192,0.535849,0.588983,0.512289,0.501521,0.588983,0.458066
1,32HC_CDS_37,3452,32HC,MAWKQPQLTDPPMVSEEIGKLNRRLLLAYAANSRAVEAGVQLHDVF...,32HC,Mycobacterium,Z,lysin_B,41216.7621,0.091384,...,0.462507,0.724050,0.724050,0.302072,0.316981,0.309322,0.430882,0.309526,0.309322,0.443445
2,39HC_CDS_42,152741,39HC,MSLALGSSGLMAAAWAAMMRLRFPGYALGRDGKPLGVDGYFGYDEE...,39HC,Mycobacterium,B6,lysin_B,35566.7986,0.084592,...,0.367217,0.277975,0.277975,0.503202,0.513208,0.550847,0.420908,0.508205,0.550847,0.439484
3,40AC_CDS_14,152741,40AC,MSLTLGSQGEIVNRWIRVMKARFVSYAGKLREDAYFGLDDAEVQKE...,40AC,Mycobacterium,A17,lysin_B,36595.5658,0.104615,...,0.648152,0.300990,0.300990,0.466580,0.535849,0.588983,0.521254,0.501214,0.588983,0.478110
4,40BC_CDS_42,152741,40BC,MSLALGSSGLMAAAWAAMMRLRFPGYALGRDGKPLGVDGYFGYDEE...,40BC,Mycobacterium,B6,lysin_B,35566.7986,0.084592,...,0.367217,0.277975,0.277975,0.503202,0.513208,0.550847,0.420908,0.508205,0.550847,0.439484


In [31]:
# Initialize LabelEncoder
le = LabelEncoder()

# Encode HostStrain and Cluster
combined_lysin_b_df['HostStrain_Encoded'] = le.fit_transform(combined_lysin_b_df['HostStrain'])
combined_lysin_b_df['Cluster_Encoded'] = le.fit_transform(combined_lysin_b_df['Cluster'])

In [32]:
combined_lysin_b_df.head()

,GeneID,phams,Name,translation,PhageID,HostStrain,Cluster,LysinType,Molecular_Weight,Aromaticity,...,Hydrophobicity_Score,Molecular_Weight_Score,Protein_Length_Score,Hamming_Distance_Score,Stability_Score,Size_Score,Similarity_Score,Treatment_Score,HostStrain_Encoded,Cluster_Encoded
0,20ES_CDS_10,152741,20ES,MSLQVGSSGELVNRWIRVMKARFASYAGKLKEDGYFGLDDKAVQQE...,20ES,Mycobacterium,A2,lysin_B,36578.3514,0.107692,...,0.229470,0.467192,0.535849,0.588983,0.512289,0.501521,0.588983,0.458066,1,7
1,32HC_CDS_37,3452,32HC,MAWKQPQLTDPPMVSEEIGKLNRRLLLAYAANSRAVEAGVQLHDVF...,32HC,Mycobacterium,Z,lysin_B,41216.7621,0.091384,...,0.724050,0.302072,0.316981,0.309322,0.430882,0.309526,0.309322,0.443445,1,144
2,39HC_CDS_42,152741,39HC,MSLALGSSGLMAAAWAAMMRLRFPGYALGRDGKPLGVDGYFGYDEE...,39HC,Mycobacterium,B6,lysin_B,35566.7986,0.084592,...,0.277975,0.503202,0.513208,0.550847,0.420908,0.508205,0.550847,0.439484,1,28
3,40AC_CDS_14,152741,40AC,MSLTLGSQGEIVNRWIRVMKARFVSYAGKLREDAYFGLDDAEVQKE...,40AC,Mycobacterium,A17,lysin_B,36595.5658,0.104615,...,0.300990,0.466580,0.535849,0.588983,0.521254,0.501214,0.588983,0.478110,1,5
4,40BC_CDS_42,152741,40BC,MSLALGSSGLMAAAWAAMMRLRFPGYALGRDGKPLGVDGYFGYDEE...,40BC,Mycobacterium,B6,lysin_B,35566.7986,0.084592,...,0.277975,0.503202,0.513208,0.550847,0.420908,0.508205,0.550847,0.439484,1,28


In [33]:
# Save to JSON
combined_lysin_b_df.to_json('combined_lysin_b_df.json', orient='records', lines=True)

# Save to CSV
combined_lysin_b_df.to_csv('combined_lysin_b_df.csv', index=False)
